In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gym-jsbsim
!pip install stable-baselines3[extra]
!pip install tensorboardX

     |████████████████████████████████| 533 kB 8.1 MB/s 
     |████████████████████████████████| 24.0 MB 1.5 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.12.1.post1 which is incompatible.
     |████████████████████████████████| 177 kB 9.3 MB/s 
     |████████████████████████████████| 1.5 MB 45.0 MB/s 
     |████████████████████████████████| 1.6 MB 49.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=bb12131e342a1910f7d086

In [3]:
import gym
import gym_jsbsim
from gym_jsbsim.catalogs.catalog import Catalog as c
import numpy as np
import math
from stable_baselines3 import DQN, A2C, DDPG, HER, PPO, SAC, TD3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

In [4]:
class Obs_TupleToBoxWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.observation_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def observation(self, obs):
        new_obs = np.empty(shape=(0,))
        for i in obs:
          new_obs = np.concatenate([new_obs,i])
        return new_obs

In [5]:
class Act_TupleToBoxWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        low = np.empty(shape=(0,))
        high = np.empty(shape=(0,))
        for i in env.action_space:
          low = np.concatenate([low,i.low])
          high = np.concatenate([high,i.high])
        self.action_space = gym.spaces.Box(low=low, high=high, dtype="float")
        
    
    def action(self, act):
#        print(act)
#        new_act = np.empty(shape=(0,))
#        for i in act:
#          new_act = np.concatenate([new_act,i])
        return act

In [6]:
env = Act_TupleToBoxWrapper(Obs_TupleToBoxWrapper(gym.make("GymJsbsim-HeadingControlTask-v0")))

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [7]:
check_env(env)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:273: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:279: UserWarning: Your action space has dtype float64, we recommend using np.float32 to avoid cast errors.
  f"Your action space has dtype {action_space.dtype}, we recommend using np.float32 to avoid cast errors."
/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [8]:
model = PPO.load("/content/drive/Shareddrives/Learn_to_fly/ppo_model_new_reward", env=env)
episode_reward = 0
state = env.reset()
print("Initial State =", state)
done = False
while not done:
  action, _ = model.predict(state, deterministic=True)
  state, reward, done, _ = env.step(action)
  episode_reward += reward
  print("action =", action, " ---> State =", state, " : Reward =", reward)
print("Simulation Time = "+"{:.3f}".format(env.get_sim_time())+" sec")
print("Episode Reward = "+"{:.3f}".format(episode_reward))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Initial State = [-3.72529030e-09  1.42108547e-14  1.81455404e-17  4.98087631e-17
 -5.06261699e-14  7.00841929e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
action = [ 0.01162816  0.03880975 -0.0858511   0.        ]  ---> State = [-3.63534950e-02 -3.60287126e-02  6.76674747e-05  7.27394505e-05
 -8.72529831e-01  7.00953551e+02  2.95664765e-03 -6.21532182e-05
  2.17359303e-02]  : Reward = 0.37335459123879816
action = [ 0.01157864  0.03878    -0.08574917  0.        ]  ---> State = [-1.43025886e-01 -1.64844876e-01  2.60481585e-05  4.55773708e-04
 -1.67477717e+00  7.01008705e+02  7.20543044e-03 -9.37078458e-04
  3.19253984e-02]  : Reward = 0.8857017018908383
action = [ 0.00986991  0.03717318 -0.08181391  0.        ]  ---> State = [-3.13306268e-01 -3.18348860e-01 -3.26459807e-05  1.20509638e-03
 -2.39975619e+00  7.01012103e+02  1.16794895e-02 -3.57134705e-05
  3.09071993e-02]  : Reward = 0.9753805939263982
ac

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


action = [-0.02403334  0.11887551 -0.07274424  0.        ]  ---> State = [-5.78072418e+00 -7.87803721e-01  2.44086377e-03  1.71746957e-02
 -9.58383049e+00  6.98730984e+02 -5.09720139e-03 -4.21963496e-02
  5.13487080e-03]  : Reward = 0.910989679073513
action = [-0.02284764  0.1221066  -0.07087727  0.        ]  ---> State = [-6.58947985e+00 -8.07963560e-01 -1.43139656e-03  1.65820651e-02
 -9.77585740e+00  6.98404908e+02 -9.76799927e-03 -5.17445026e-02
  4.61871044e-03]  : Reward = 0.8849912242259329
action = [-0.02048217  0.12529388 -0.06654938  0.        ]  ---> State = [-7.40143825e+00 -8.23370397e-01 -5.92418875e-03  1.56699974e-02
 -9.65816914e+00  6.98080517e+02 -1.22982323e-02 -5.60754737e-02
  3.12441544e-03]  : Reward = 0.8570853931807739
action = [-0.0167119   0.12933365 -0.05924684  0.        ]  ---> State = [-8.19060504e+00 -8.29656974e-01 -1.06107983e-02  1.46226869e-02
 -9.23169480e+00  6.97757667e+02 -1.25439127e-02 -5.56220278e-02
  5.69246779e-04]  : Reward = 0.8293185855

In [9]:
model = PPO.load("/content/drive/Shareddrives/Learn_to_fly/ppo_model", env=env)
episode_reward = 0
state = env.reset()
print("Initial State =", state)
done = False
while not done:
  action, _ = model.predict(state, deterministic=True)
  state, reward, done, _ = env.step(action)
  episode_reward += reward
  print("action =", action, " ---> State =", state, " : Reward =", reward)
print("Simulation Time = "+"{:.3f}".format(env.get_sim_time())+" sec")
print("Episode Reward = "+"{:.3f}".format(episode_reward))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Initial State = [-3.72529030e-09  1.42108547e-14  1.81455404e-17  4.98087631e-17
 -5.06261699e-14  7.00841929e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
action = [-0.01933245  0.04399324 -0.02767818  0.55144894]  ---> State = [-3.66293564e-02 -1.17252210e-02  3.77019779e-05 -2.12696066e-04
 -8.79801028e-01  7.00951900e+02 -8.12271313e-03 -1.24717875e-03
  7.00581420e-03]  : Reward = 0.917503710263639
action = [-0.01681887  0.04418482 -0.02468718  0.54661536]  ---> State = [-1.44050810e-01 -5.21496780e-02 -1.48676534e-04 -1.13650289e-03
 -1.68365181e+00  7.01007928e+02 -1.48068749e-02 -3.50088523e-03
  9.55111887e-03]  : Reward = 0.9797072736130275
action = [-0.01270593  0.04681893 -0.02061325  0.55440676]  ---> State = [-3.14885903e-01 -9.62444432e-02 -4.81254923e-04 -2.48712037e-03
 -2.40174696e+00  7.01015586e+02 -1.76046775e-02 -4.67400053e-03
  8.17312705e-03]  : Reward = 0.9789596719098076
acti

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


action = [ 0.02476758  0.09463109 -0.0008547   0.8216188 ]  ---> State = [-5.15843920e+00 -3.92437212e-02 -1.65782501e-02  5.44286941e-02
  1.83920006e-01  7.04601753e+02  4.00638172e-02  1.57786972e-02
  1.44952035e-03]  : Reward = 0.9657320446401791
action = [ 0.0246965   0.09449887 -0.00093832  0.82106864]  ---> State = [-5.13720835e+00 -5.08593642e-02 -1.52120472e-02  5.77783633e-02
  3.06580518e-01  7.04771799e+02  4.04636418e-02  1.70703654e-02
  1.51242878e-03]  : Reward = 0.9601467426209425
action = [ 0.02463407  0.09441854 -0.00099345  0.82070112]  ---> State = [-5.11044472e+00 -6.31626772e-02 -1.38082210e-02  6.11573829e-02
  3.18035133e-01  7.04938104e+02  4.07464340e-02  1.63375580e-02
  1.57687862e-03]  : Reward = 0.9543381601865683
action = [ 0.02461073  0.09439617 -0.00101008  0.82058823]  ---> State = [-5.08694547e+00 -7.56246697e-02 -1.25259308e-02  6.45572043e-02
  2.31337155e-01  7.05100893e+02  4.09450928e-02  1.38966010e-02
  1.65080413e-03]  : Reward = 0.948842457